In [1]:
# from MuyGPyS import config
# config.update("muygpys_jax_enabled", False)

import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from MuyGPyS.examples.classify import do_classify
from MuyGPyS.gp.deformation import F2, Isotropy, l2
from MuyGPyS.gp.hyperparameter import Parameter, Parameter as ScalarParam
from MuyGPyS.gp.kernels import RBF, Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.optimize import Bayes_optimize, L_BFGS_B_optimize
from MuyGPyS.optimize.loss import LossFn, cross_entropy_fn, looph_fn



/home/eleh/miniconda3/envs/muygps/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:

# data_path = ['norm_11.csv','norm_1.csv', 'norm_21.csv', 'raw_image_data.csv']
data_path = [ 'raw_image_data.csv']
# get rid of "../data/data-norm/"
norm_data_names = data_path
norm_data_names

['raw_image_data.csv']

In [3]:
def generate_onehot_value(values):
    onehot = []
    for val in values:
        if val == 0:
            onehot.append([1., -1.])
        elif val == 1:
            onehot.append([-1., 1.])
    return onehot

In [4]:
nn_kwargs_exact = {"nn_method": "exact", "algorithm": "ball_tree"}

nn_kwargs_hnsw = {"nn_method": "hnsw"}

k_kwargs_rbf ={
            "kernel": RBF(
                 deformation=Isotropy(
                     metric=F2,
                 length_scale=Parameter(1.0, (1e-2, 1e2)),
                 ),
            ),
            "noise": HomoscedasticNoise(1e-5),
            }
k_kwargs_mattern= { "kernel": Matern(
             smoothness=ScalarParam(0.5),
             deformation=Isotropy(
                 metric=F2,
                 length_scale=Parameter(1.0, (1e-2, 1e2)),
             ),
         ),
         "noise": HomoscedasticNoise(1e-5),
         }

### Cross Validation
We use Monte-Carlo Cross Validation to investigate the variability in the performance of MuyGPs with respect to the raw data in comparison to the normalized data.

In [5]:

nn_kwargs_hnsw = {"nn_method": "hnsw"}

# k_kwargs_mattern= { "kernel": Matern(
#              smoothness=ScalarParam(0.858571),
#              deformation=Isotropy(
#                  metric=F2,
#                  length_scale=Parameter(.756327, (1e-2, 1e2)),
#              ),
#          ),
#          "noise": HomoscedasticNoise(0.244898),
#          }

k_kwargs_mattern= { "kernel": Matern(
             smoothness=ScalarParam(0.5),
             deformation=Isotropy(
                 metric=l2,
                 length_scale=Parameter(10.0, (1e-2, 1e2)),
             ),
         ),
         "noise": HomoscedasticNoise(1e-6),
         }

In [8]:
import pickle

m = 100 # cross validation size
train_sizes = [0.8 for i in range(m)]
# read in data
# path = 'norm_21.csv'
# path = 'raw_image_data.csv'
path = 'nthroot_0.3103.csv'
path1 = '../../data/data-norm/max-only/' + path
data = pd.read_csv(path1,na_values='-')
data.fillna(0,inplace=True)
data_label = ''.join(path.split('.')[:2])
truth_labels = data.iloc[:, 0].values
image_data = data.iloc[:, 1:].values

accuracy = {'raw-data':[], 'rroot-03103':[]}
# i = 0   # run one at a time since MUYGPS won't left me run multiple at once
for i,size in enumerate(train_sizes):
    X_train, X_test, y_train, y_test = train_test_split(image_data, truth_labels, train_size=size)


    print("=============== ", data_label, " ===============")
    print('Training data:', len(y_train[y_train==0]), 'single stars and', len(y_train[y_train==1]), 'blended stars')
    print('Testing data:', len(y_test[y_test==0]), 'single stars and', len(y_test[y_test==1]), 'blended stars')

    onehot_train, onehot_test = generate_onehot_value(y_train), generate_onehot_value(y_test)

    train = {'input': X_train, 'output': onehot_train, 'lookup': y_train}
    test = {'input': X_test, 'output': onehot_test, 'lookup': y_test}

    print("Running Classifier on", data_label)
    #Switch verbose to True for more output


    muygps, nbrs_lookup, surrogate_predictions = do_classify(
                                test_features=np.array(test['input']), 
                                train_features=np.array(train['input']), 
                                train_labels=np.array(train['output']), 
                                nn_count=35,
                                batch_count=200,
                                loss_fn=looph_fn,
                                opt_fn=L_BFGS_B_optimize,
                                k_kwargs=k_kwargs_mattern,
                                nn_kwargs=nn_kwargs_hnsw,
                                verbose=False)
    predicted_labels = np.argmax(surrogate_predictions, axis=1)
    accur = np.around((np.sum(predicted_labels == np.argmax(test["output"], axis=1))/len(predicted_labels))*100, 3)
    if path == 'raw_image_data.csv':
        accuracy['raw-data'].append(accur)
    else:
        accuracy['rroot-03103'].append(accur)
    print(f"Total accuracy at Iter {i+1}: {accur}%")

    # check if accuracy file exists, if not save it as a new file and if it does exist, append to it
    if path == 'raw_image_data.csv':
        saveto = './cv-results/raw-data-cv-results.pkl'
    else:
        saveto = './cv-results/rroot-03103-cv-results.pkl'
    try:
        with open(saveto, 'rb') as f:
            acc = pickle.load(f)
            acc.update(accuracy)
        with open(saveto, 'wb') as f:
            pickle.dump(acc, f)
    except:
        with open(saveto, 'wb') as f:
            pickle.dump(accuracy, f)


===============  nthroot_03103  ===============
Training data: 12137 single stars and 9664 blended stars
Testing data: 2972 single stars and 2479 blended stars
Running Classifier on nthroot_03103
Total accuracy at Iter 1: 82.26%
===============  nthroot_03103  ===============
Training data: 12084 single stars and 9717 blended stars
Testing data: 3025 single stars and 2426 blended stars
Running Classifier on nthroot_03103
Total accuracy at Iter 2: 82.774%
===============  nthroot_03103  ===============
Training data: 12072 single stars and 9729 blended stars
Testing data: 3037 single stars and 2414 blended stars
Running Classifier on nthroot_03103
Total accuracy at Iter 3: 81.343%
===============  nthroot_03103  ===============
Training data: 12111 single stars and 9690 blended stars
Testing data: 2998 single stars and 2453 blended stars
Running Classifier on nthroot_03103
Total accuracy at Iter 4: 82.554%
===============  nthroot_03103  ===============
Training data: 12107 single stars

In [7]:
# pickle save accuracy
import pickle
with open(saveto, 'wb') as f:
    pickle.dump(accuracy, f)
